In [ ]:
import os
import numpy as np
import pydicom
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

def get_dicom_files_from_folder(folder_path):
    dicom_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.dcm')]
    return sorted(dicom_files)  # Sorting ensures correct slice order

def load_dicom_stack(dicom_files):
    slices = [pydicom.dcmread(file) for file in dicom_files]
    slices.sort(key=lambda x: float(x.ImagePositionPatient[2]) if hasattr(x, "ImagePositionPatient") else x.InstanceNumber)
    pixel_stack = np.stack([s.pixel_array.astype(np.float32) for s in slices])
    return pixel_stack

def simulate_bSSFP(dicom_files_t1, dicom_files_t2, flip_angle=25, TR=12):
    t1_stack = load_dicom_stack(dicom_files_t1)
    t2_stack = load_dicom_stack(dicom_files_t2)

    # Restrict slices to specific range
    slice_start, slice_end = 129, 130
    slice_indices = np.linspace(slice_start, slice_end, 1, dtype=int)

    # Initialize lists to store S_maps and T2/T1 ratios for different T1/T2 maps
    smap_variations = []
    ratio_variations = []

    for slice_idx in slice_indices:
        # Extract the specific slice
        t1_slice = t1_stack[slice_idx, :, :]
        t2_slice = t2_stack[slice_idx, :, :]

        # Define three different T1 and T2 map settings
        t1_t2_variations = [
            (400 + (1600 * t1_slice), 0.4 + (4 * t2_slice)),   # Variation 1
            (400 + (1600 * t1_slice), 40 + (40 * t2_slice)),   # Variation 2
            (400 + (1600 * t1_slice), 400 + (1000 * t2_slice)),  # Variation 3
        ]

        # Calculate S_maps and T2/T1 ratios for each variation
        smap_set = []
        ratio_set = []
        for T1_map, T2_map in t1_t2_variations:
            M0_map = t1_slice  # Unnormalized intensity as M0 proxy
            E1_map = np.exp(-TR / T1_map)
            E2_map = np.exp(-TR / T2_map)
            alpha = np.radians(flip_angle)
            S_map = (
                M0_map * np.sqrt(E2_map * (1 - E1_map)) * np.sin(alpha) /
                (1 - (E1_map - E2_map) * np.cos(alpha) - E1_map * E2_map)
            )
            smap_set.append(S_map)
            ratio_set.append(np.mean(T2_map / T1_map))  # Average T2/T1 ratio for the slice
        smap_variations.append(smap_set)
        ratio_variations.append(ratio_set)

    # Plotting S_maps with T2/T1 ratio in titles
    fig, axes = plt.subplots(len(slice_indices), 3, figsize=(20, 15))
    axes = axes.flatten() if isinstance(axes, np.ndarray) else [axes]

    for i, slice_idx in enumerate(slice_indices):
        for j, (S_map, ratio) in enumerate(zip(smap_variations[i], ratio_variations[i])):
            ax = axes[i * len(smap_variations[0]) + j]
            ax.imshow(S_map, cmap='gray')
            ax.set_title(f"T2/T1 Ratio: {ratio:.3f}", fontsize=40)
            ax.axis('off')

    plt.tight_layout()
    plt.show()


t1_folder = "/content/drive/MyDrive/T1_Weighted/FLASH_3D_stitched"
t2_folder = "/content/drive/MyDrive/T2_Weighted/TSE_3D_stitched"

dicom_files_t1 = get_dicom_files_from_folder(t1_folder)
dicom_files_t2 = get_dicom_files_from_folder(t2_folder)

simulate_bSSFP(dicom_files_t1, dicom_files_t2)